# Generate book table of contents

Here we generate the Jupyter book table of contents file at `docs/_toc.yml`.

In [ ]:
# Notebook parameters. Values here are for development only and
# will be overridden when running via snakemake and papermill.

config_file = "../../../config/agam.2025.03.05.yaml"

In [ ]:
from bokeh.io import output_notebook
import yaml
from selection_atlas.setup import AtlasSetup
from selection_atlas.page_utils import AtlasPageUtils

# Initialise the atlas setup.
setup = AtlasSetup(config_file)
page_utils = AtlasPageUtils(setup=setup)

# N.B., do not add the "remove-output" tag to this cell!!! If you do,
# the bokeh javascript libraries will not get loaded in the generated
# HTML page. The call to output_notebook() injects javascript in the
# cell output which triggers the bokeh javascript libraries to be loaded
# in the page.
output_notebook(hide_banner=True)

In [ ]:
countries = page_utils.gdf_cohorts["country_alpha2"].unique()
countries

In [ ]:
alerts_part = dict(
    caption="Selection alerts",
    chapters=[
        dict(
            file="alerts",
            sections=[dict(file=f"alert/{a}") for a in setup.alerts],
        ),
    ],
)
alerts_part

In [ ]:
contig_part = dict(
    caption="By chromosome",
    chapters=[
        dict(
            file=f"contig/{contig}",
        )
        for contig in setup.contigs
    ],
)
contig_part

In [ ]:
def build_country_chapter(country):
    cohorts = sorted(
        page_utils.gdf_cohorts.query(f"country_alpha2 == '{country}'")["cohort_id"]
    )
    chapter = dict(
        file=f"country/{country}",
        sections=[
            dict(
                file=f"cohort/{cohort}",
            )
            for cohort in cohorts
        ],
    )
    return chapter

In [ ]:
countries_part = dict(
    caption="By country",
    chapters=[build_country_chapter(country) for country in countries],
)
countries_part

In [ ]:
about_part = dict(
    caption="About",
    chapters=[
        dict(file="methods"),
        dict(file="faq"),
        dict(file="glossary"),
    ],
)
about_part

In [ ]:
toc = dict(
    format="jb-book",
    root="index",
    parts=[
        alerts_part,
        contig_part,
        countries_part,
        about_part,
    ],
)
toc

In [ ]:
with open(setup.jb_source_dir / "_toc.yml", mode="w") as output_file:
    yaml.safe_dump(toc, output_file)